# Theory behind `PyDEns`

Welcome! This tutorial explains math behind `PyDEns`, introducing the very details of equation solving. Our library is heavily inspired by the paper [DGM: A deep learning algorithm for solving partial differential equations](http://arxiv.org/abs/1708.07469), but is capable of doing way more: it is applicable to parametric equations, to equations with uncertainty, and can even be used to inforce prior information on solution via adjusting imperfect coefficients. This notebook introduces basic concepts, and is structed as follows:

* [Why do we use Neural Networks?](#why)
* [PDE problem setup](#problem)
* [Galerkin Method](#galerkin)
* [Neural Networks as approximators](#theorems)
* [Deep Galerkin method](#dg)
* [Tracting the intractable](#sampling)
* [Ansatz](#ansatz)
* [Advanced topics](#advanced)

We do recommend to check out this notebook; yet, if you want to go straight to PDE solving, you can go right to [the next tutorial](1.%20Basic%20PDE%20solving.ipynb).

<a id='why'></a>
## Why do we use Neural Networks?

Before diving deep into the realm of deep learning, we need to understand exactly the reasons, why we can't be satisfied with more traditional methods. After all, *finite element*, *finite differences*, and many others approaches are under active development for the last hundred of years, and should meet all the criteria of a good PDE solver, right? Not exactly. There are quite a few problems, related to both approaches itself and their software implementations:

* most of popular numerical techniques operate on a fixed mesh. That greatly simplifies some of calculations and gives rigid structure, yet detrimental when solving problems that require very fine granularity: number of computations growth exponentially on cell size

* working with uncertainty and parameters is both ineffective and ugly

* in the recent years, accelerators like GPU and TPU have improved tremendously, yet most of PDE solving software does not utilize this potential to its full extent

We can solve most of these problems by simply borrowing modern tensor-processing frameworks like `TensorFlow` or `PyTorch`: they allow us to seamlessly integrate most of GPUs/TPUs into our workflow, greatly increasing the speed of inference.

Being mesh-free is useful not only with regards to finding solution on a fine grid: it allows us to incorporate prior information directly into the learning process. We can easily 'tell' algorithm on which areas it should perform best, and also point out places that we don't care too much about (again, saving precious computational resources).

At the first glance it might seem that NN usage in such classical problem of mathematics goes against centuries of progress in that domain. We will shortly see that, in fact, neural network-based approximators fit very harmoniously into the ensemble of existing approaches, enjoying various tricks both from emerging field of machine learning and well-estabilished theory of differential equations.

<a id='problem'></a>
## PDE problem setup

We are set to find function $u(x, t)$ with respect to:

$$ F\left(u; t, x; \nabla u; \nabla^2 u; ...\right) = 0, $$

where $x$ is a $n$-dimensional vector, $t$ is time variable, which can be absent, and $F$ is some general functional. We use $\nabla u$, $\nabla^2 u$ to denote set of partial derivatives of the function of the first/second order respectively. For now, we would constraint $F$ to belong to a class of continuous functions, yet, as we would see, that is not strictly necessary.

In order to make the solution unique, we must impose ***boundary*** and/or ***initial*** conditions

$$ \left. u(x, t) \right\rvert_{x \in \partial\Omega} = g(x, t), $$

$$  u(x, t_0) = u_0(x), $$

$$  \left. \frac{\partial u(x, t)}{\partial t} \right\rvert_{t = t_0} = u_1(x), $$

with $\Omega$ being part of $\mathcal{R}^{n+1}$.

**Note:** we also use $F[u]$ for brevity.

<a id='galerkin'></a>
## Galerkin method

It is time to revisit one of the traditional methods of solving functional (not only differential) equations. The idea behind this approach is to approximate the unknown function with a finite linear combination of pre-defined basis functions. In order to achieve the best in-class approximation, we select coefficients of linear combination that minimize some error function. To be more rigorous, we swap unknown function $u(x, t)$ with

$$ \widehat{u}(x, t) = \sum_{i=0}^N \alpha_i\phi_i(x, t).$$

We define error at one space point as 

$$ J(x, t; \alpha) = \left\| F[\widehat{u}]\right\| + \left\|\widehat{u}(x, t) - g(x, t) \right\|_{x \in \partial\Omega} +  \left\|\widehat{u}(x, t) - u_0(x) \right\|_{t = t_0}, $$

and in order to obtain optimal values for $\alpha_i$ we must find $argmin$ of the integral over entire domain

$$ \int_{\Omega} J(x, t; \alpha) dx dt.$$

Note that in most cases various families of orthogonal polynomials are used to form the basis. That is due to polynomials being [dense](https://en.wikipedia.org/wiki/Stone%E2%80%93Weierstrass_theorem) in the space of continuos functions, which allows them to approximate them arbitrarily well (with any desired precision).

There are quite a few different ways to find optimal $\alpha$ values: some of them are analytical, some of them are numerical. We will look at them closely later in regards to neural network approximation.

<a id='theorems'></a>
## Neural Networks as approximators

Now we need to know some underlying theory behind the representational power of neural networks. A well-known [result](https://en.wikipedia.org/wiki/Universal_approximation_theorem) states, that any real-valued continuos function can be approximated as closely as desired on a compact by a feed-forward neural network with a single hidden layer, finite number of neurons and nonconstant continuos bounded activation function. In other words, feed-forward neural networks are dense in the space of continuos functions (just like polynomials!).

It is worth noting, though, that the number of neurons needed to achieve error rate of $\epsilon$ is exponential in the dimensionality of approximated function. On the other hand, it [has been shown](https://arxiv.org/abs/1908.09375), that functions that can be represented as composition of functions with lesser dimensionality, can achieve desired tolerance with a much lower size.

All that allows us to say that, from approximation point of view, neural networks are not that different from polynomials: instead of using ***degree*** as the sizing parameter of a polynomial, we use ***number of parameters (neurons)*** in the network for quite similar purposes, and enjoy its ability to closely follow any continuos real-valued function. 

<a id='dg'></a>
## Deep Galerkin method

As we now know, we can use neural network with set of parameters $\theta$ as approximator. We can use multiple such networks as basis for ***Galerkin method***, but, actually, we can just use one and increase the size of it to control its representational power. Thus, our approach looks very simple:

$$ \widehat{u}(x, t) = Net(x, t; \theta).$$

Point-wise error functional, as well as domain-wide one, as before, are given by

\begin{gather}
    J(\theta) = \left\|F[Net]\right\| + \left\|Net(x, t; \theta) - g(x, t) \right\|_{x \in \partial\Omega} + \\ +  \left\|Net(x, t; \theta) - u_0(x) \right\|_{t = t_0},
\end{gather}

$$\int_{\Omega} J(x, t; \theta) dx dt. $$ 

We should minimize value of the error over  NN parameters $\theta$ to achieve optimal neural networks weights. 

Before talking about actual minimization, we should look at even more apparent issue: in order to compute error (even for one point in space), we must evaluate $F[Net]$. Operator $F$, as was defined before, contains derivatives of up to the second order of the inputs, which can cumbersome to calculate even for the simplest functions. We are potentially talking about neural networks with hundreds of layers, so how do we do that?

Fortunately, we can use the same concepts that power up modern deep learning: [computational graphs and backpropogation](https://colah.github.io/posts/2015-08-Backprop/). Usually they are used to compute gradients of the network output with respect to its weights in order to adjust them to the task; we use this capability both to make the approximation better, as well as calculate values of differential form. If you are not familiar with these concepts, we highly recommend watching [this video](https://www.youtube.com/watch?v=-yhm3WdGFok).

<a id='sampling'></a>
## Tracting the intractable

The error function, given by an integral, is rarely tractable. It means that we can't just evaluate its value, so we swap the integral over continuos domain with summation over finite ***batch*** of points:

$$ \int_{\Omega} J(x, t; \theta)  dx dt \longrightarrow \sum_{x^i, t^i \in B \in \Omega} J(x^i, t^i; \theta). $$

Note that we fully control the entire process of generating points from $\Omega$: we can sample more points in promising regions, completely ignoring uninteresting ones. On the other hand, shall we need it, we can simulate any fixed grid: for example, it seems reasonable to sample points near boundaries/zero-time at the very beginning of the learning process, so we can mimic some conventional numerical methods. Even during the actual inference we can *fine-tune* the network on the fly right before using it to obtain approximation of the solution.

**Note:** with great power comes great responsibility. Such massive freedom requires elaborate framework, and that is exactly what `PyDEns` is for!

<a id='ansatz'></a>
## Ansatz

As was said before, we can use not only modern techniques for training complex neural networks, but also some tricks from more classical domains of mathematics. One of them is ansatz, which allows to explicitly bind initial and/or boundary conditions.

Taking a look once again at 
\begin{gather}
    J(\theta) = \left\|F[Net]\right\| + \left\|Net(x, t; \theta) - g(x, t) \right\|_{x \in \partial\Omega} + \\ +  \left\|Net(x, t; \theta) - u_0(x) \right\|_{t = t_0},
\end{gather}
we can see that in consists of multiple equal terms, so optimization process treats them, well, equally. Sometimes that can be detrimental: conditions must be satisfied exactly, with zero tolerance. Ansatz allows us to achieve such binding by applying a simple yet effective transformation to the approximator:

$$ Net \longrightarrow A(Net, g, u_0, u_1). $$

Exact form of function $A$ depends on number of conditions to inforce. For example, in the simplest possible case, where $t_0=0$ and we only have condition on function in the initial time ($u_0$), we can use

$$ A = u_0(x) + t \cdot Net(x, t; \theta)$$

to explicitly enforce conditions on our approximator

<a id='history'></a>
## History

In 1998, [Lagaris et al.](https://pdfs.semanticscholar.org/5ebb/e0b1a3d7a2431bbb25d6dfeec7ed6954d633.pdf) proposed to approximate PDE solutions with shallow neural networks. Unfortunately, backprop was not popular back then, so authors had to manually compute derivatives of network w.r.t. inputs. Due to complexities of the method and lack of evidence of practical benefits, this domain of research took a long pause.

After development of convinient machine learning libraries like [TensorFlow](https://ai.google/research/pubs/pub45381), the interest has emerged once again. Most of the modern approaches can be traced back to [DeepGalerking by Sirignano and Spiliopoulos, 2018](https://arxiv.org/abs/1708.07469), which heavily inspired our framework.

<a id='advanced'></a>
## Advanced topics
Proposed method readily extends to the case of multiple equations with multiple unknowns. Yet, a more exciting avenue is solving PDEs with uncertainties, parameters or to even propose novel settings of PDE solving. You can learn more about all these things [here](2.%20Advanced%20PyDEns.ipynb), but be sure to start with the [basics](1.%20Basic%20PDE%20solving.ipynb)!